## Download patches from planet using GEE

Based on https://gist.github.com/gorelick-google/4c015b79119ef85313b8bef6d654e2d9

In [ ]:
import ee
import logging
import multiprocessing
import requests
import shutil
import backoff
from retry import retry
from pathlib import Path

ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
@retry(tries=10, delay=1, backoff=2)
def getResult(_, feature):
    """Handle the HTTP requests to download an image."""

    # Generate the desired image from the given point.
    region = ee.Feature(feature).geometry()
    name = feature["properties"]["id"]
    
    image = (
        planet_afk.filter(ee.Filter.date("2019-01-01", "2019-12-31"))
          .mean()
    ).clip(region)
    
    # Fetch the URL from which to download the image.
    url = image.getDownloadURL({"scale": 4.77, "region":image.geometry(), "format":"GEO_TIFF"})
    
    # Handle downloading the actual pixels.
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    file_path = (Path.home()/"1_modules/4_MultiLabel/train-tif-ce/fn").with_name(f"{name}.tif")
    
    with open(file_path, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)
    
    print("Done: ", name)


In [ ]:
table = ee.FeatureCollection("projects/planet-afk/assets/kenya_points2")
planet_afk = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa")
plots = table.map(lambda center: center.buffer(70.710).bounds())

In [ ]:
# This is an awful way to extract the features, but it works
# I didn't want to create the geometries in pandas, I wanted to use GEE.
f5000 = ee.FeatureCollection(plots.toList(5000)).getInfo()
f5000_10000 = ee.FeatureCollection(plots.toList(5000, 5000)).getInfo()
f10000_10810 = ee.FeatureCollection(plots.toList(810, 10000)).getInfo()
f5000["features"] = f5000["features"]+f5000_10000["features"]+f10000_10810["features"]
features = f5000["features"].copy()
features[:5]

In [ ]:
if __name__ == '__main__':
    
    logging.basicConfig()
    pool = multiprocessing.Pool(25)
    pool.starmap(getResult, enumerate(features))
    pool.close()